In [6]:
import requests
import json 
from pprint import pprint
import pandas as pd
import time

from pymongo import MongoClient

HOST = 'cluster0.zsn5y.mongodb.net'
USER = 'elije465'
PASSWORD = '465684'
DATABASE_NAME = 'LOL'
COLLECTION_NAME = 'puuid'
MONGO_URI = f"mongodb+srv://{USER}:{PASSWORD}@{HOST}/{DATABASE_NAME}?retryWrites=true&w=majority&ssl=true&ssl_cert_reqs=CERT_NONE"
client = MongoClient(MONGO_URI) #몽고 클라이언트(라이브러리) 몽고URI를 가져올수있게 도와줌 에 몽고 URI를 연결한다
database=client[DATABASE_NAME]#DATABSE, 연결된 클라이언트에 데이터베이스를 연결
collection=database[COLLECTION_NAME]#TABLE, 연결된 데이터베이스에 콜렉션을 연결
coll_sid=database['sid']
coll_matchid=database['matchid']


api='RGAPI-17ff87d6-917c-45d7-a415-ce5256a3946f' #내 API다 계속 갱신해줘야댐 elije46 100823 만료
api2='RGAPI-f0c2288a-d0be-4c9e-b47e-126c12644f26'#elije456 100913 만료
api3='RGAPI-3e05e2e3-8413-484c-a6bc-e3c8328aede9' #elije2 100908만료
ranker=['challengerleagues','masterleagues','grandmasterleagues']
normal_tier=['DIAMOND','PLATINUM','GOLD','SILVER','BRONZE','IRON']
leagues = ['I','II','III','IV']

class search:
    def __init__(self,tier,page=1,league='I'):  #,sid_list=[]
        self.tier=tier
        self.page=page
        self.league=league
        #self.sid_list=sid_list

    def search_sid_puuid(self,summonname):  
        """모아둔 소환사명을 이용해서 puuid를 모은다 
        인수로 소환사명을 넣게됨
        puuid 200개 200초이상걸림..ㅠㅠ 2계정돌려도
        """  
        sn=summonname['summonerName']
        puuid1=set()    
        for i in range(len(sn)):
            id=sn.iloc[i]
            try:
                
                searchid=f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{id}?api_key={api}"
                r=requests.get(searchid)    
                if r.status_code == 429: 
                    searchid2=f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{id}?api_key={api2}"
                    r2=requests.get(searchid2)
                    while r2.status_code == 429:
                        time.sleep(120)
                        searchid2=f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{id}?api_key={api2}"
                        r2=requests.get(searchid2)
                        break   
                    r=r2
                
                puuid1.add(r.json()['puuid'])  
                ##########
                collection.insert_one(document=r.json())
                print('send puuid')
                
            except:pass       
        puuid=pd.DataFrame(puuid1)         
        return puuid

    def search_tier_sid(self):   
        """
        인수로 티어와 리그번호 페이지를 입력하면 등록된 소환사들을 보여준다, 이 리스트를 기반으로 소환사명을 모은다 
        tier : DIAMOND,PLATINUM,GOLD,SILVER,BRONZE,IRON
        leagues : I,II,III,IV
        top_tier : grandmasterleagues,challengerleagues,masterleagues
        탑티어 입력시 다른 인자는 필요없다
        """  
        def get_df():  
            r=requests.get(search_tier)              
            while r.status_code == 429:
                time.sleep(120)
                r=requests.get(search_tier)   
            coll_sid.insert_one(document=r.json())
            print('send sid') 
            df=pd.DataFrame(r.json())
            return df
        
        if self.tier in ranker:
            search_tier=f"https://kr.api.riotgames.com/lol/league/v4/{self.tier}/by-queue/RANKED_SOLO_5x5?api_key={api}"
            df=get_df()
            df=pd.DataFrame(dict(df['entries'])).T   
            return df
        elif self.tier in normal_tier:
            search_tier=f"https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{self.tier}/{self.league}?page={self.page}&api_key=RGAPI-7815325f-9e51-406c-b780-b5146f136879"                    
            return get_df()     

    def save_sid_puuid(self):        
        """
        sid 찾기와 puuid찾기를 같이 실행해주고 tier.csv로 저장해준ㄴ다
        """        
        
        def saver():
            sid=self.search_tier_sid() #sid 가져오기            
            puuid=self.search_sid_puuid(sid) #sid로 puuid 가져오기
            sid['puuid']=puuid
            sid.to_csv(f'{self.tier}+{self.league}+{self.page}.csv',index=False,encoding='cp949') #csv 로 빼둠
            return sid
        
        if self.tier in ranker:
            saver()
        elif self.tier is ranker:
            for x in self.tier:
                self.tier=x
                saver()
        else: 
            for z in self.tier:
                self.tier=z
                for x in self.league:                
                    self.league=x                
                    for y in range(1,200):    
                        self.page=y                
                        emptydf=saver()
                        if emptydf.empty:
                            print('DataFrame is empty!')
        return print('Finish~')    

def search_matchid(puuid,name):    
    """
    puuid로 매치아이디를 모은다!
    
    """     

    matchid1=[]    
    for i in puuid:     
       
    # for i in range(len(puuid)):        
    #     id=puuid.iloc[i]
        try:
            search_matchid=f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{i}/ids?type=ranked&start=0&count=100&api_key={api}" 
            r=requests.get(search_matchid)
            
          
            while r.status_code == 429:
                time.sleep(120)
                search_matchid=f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{i}/ids?type=ranked&start=0&count=100&api_key={api}" 
                r=requests.get(search_matchid)
            
            coll_matchid.insert_one({'matchid':r.json()})   
            print('send matchid') 
            matchid1.append(r.json())              
        except:print('fail')
    matchid2=set(sum(matchid1,[]))
    matchid=pd.DataFrame(matchid2) 
    matchid.to_csv(f'{name}+{"matchid"}.csv',index=False,encoding='cp949') #csv 로 빼둠 
    return       



In [4]:
ranker_list=search(ranker) #랭커들 sid,puuid 저장
ranker_list.save_sid_puuid()

# normal_list=search(normal_tier)
# normal_list.save_sid_puuid() #저티어 아이디빼기


send sid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
send puuid
s

In [4]:
puuid_db=collection.find() #몽고db에서 데이터가져오기

In [8]:
cdf=pd.read_csv('challengerleagues+1.csv',encoding='cp949')
gdf=pd.read_csv('grandmasterleagues+1.csv',encoding='cp949')
mdf=pd.read_csv('masterleagues+1.csv',encoding='cp949')
gmc=pd.concat([cdf,gdf,mdf])
gmc=gmc.dropna()
gmc['puuid']


0       8Pcp6RmpKXWo5VsxWdqDHE0etdA8JA_-FHJqzS-sLS7-UG...
1       YWXrXmmN883qw8MXPZOZGx-dNGjQXPafWJuzQHP01JNBE8...
2       DMsr5d4N_oDlUuZmp4N00HeLCqsg_VAqcEp5uV06dmB74g...
3       dWbGAs5kwitLcPO8oOzJeItb_wKXw8OgJZ7jhw-h8HiQBv...
4       Ns_7gqcPz2C7GEBh2x1ptz38zHrBdfje6trYTC_yzeicF8...
                              ...                        
3332    WhI3Ujm1yw9dIjPVk-ZVHVZnczTBQJgHZk5KlsqrmLWati...
3333    Dyd9pFcvAqHWxHUrnKs-d8uuNJP5ZyyJ9RwIE1BV_nSCc5...
3334    zI40H1ESCVOB9fKhpiOcH7YbCkgw1YKBqLpTl-ZVYye1BQ...
3335    975bmVwNdkEy1JR54ieQHdLLEqEgHyKJaWl1bn6FgjPTXF...
3336    bQ_tJG5FBRR2AmmEKnbV1XakouKaBLmVDMc0fRyvNgVtEm...
Name: puuid, Length: 4331, dtype: object

In [9]:
search_matchid(gmc['puuid'],'gmc_matchid')#매치아이디 가져오기


send matchid
send matchid
